In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [18]:
apps = pd.read_csv('apps.csv')
jobs = pd.read_csv('jobs.csv')
users = pd.read_csv('users.csv')

In [19]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22133 entries, 0 to 22132
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   UserID                22133 non-null  int64  
 1   WindowID              22133 non-null  int64  
 2   Split                 22133 non-null  object 
 3   City                  22133 non-null  object 
 4   State                 22133 non-null  object 
 5   Country               22133 non-null  object 
 6   ZipCode               22104 non-null  object 
 7   DegreeType            22133 non-null  object 
 8   Major                 22133 non-null  object 
 9   GraduationDate        17491 non-null  object 
 10  WorkHistoryCount      22133 non-null  int64  
 11  TotalYearsExperience  22133 non-null  float64
 12  CurrentlyEmployed     22133 non-null  object 
 13  ManagedOthers         22133 non-null  object 
 14  ManagedHowMany        22133 non-null  int64  
dtypes: float64(1), int6

In [20]:
users['DegreeType'] = users['DegreeType'].fillna('')
users['Major'] = users['Major'].fillna('')
users['TotalYearsExperience'] = users['TotalYearsExperience'].fillna('').astype(str)

users['DegreeType'] = users['DegreeType'] + ' ' + users['Major'] + ' ' + users['TotalYearsExperience']


In [21]:
# Define the path where you want to save the new CSV file
output_csv_path = 'modified_users.csv'

# Save the modified users DataFrame to a new CSV file for the UI data
users.to_csv(output_csv_path, index=False)

In [22]:
users.head()

,UserID,WindowID,Split,City,State,Country,ZipCode,DegreeType,Major,GraduationDate,WorkHistoryCount,TotalYearsExperience,CurrentlyEmployed,ManagedOthers,ManagedHowMany
0,13,6,Test,Philadelphia,PA,US,19143,Bachelor's Psychological & Social Sciences 5.0,Psychological & Social Sciences,2011-12-01 00:00:00,6,5.0,Yes,No,0
1,64,6,Train,Columbus,OH,US,43230,Master's Business Administration and Managemen...,"Business Administration and Management, General",2011-12-01 00:00:00,3,22.0,Yes,No,0
2,101,6,Train,Brick,NJ,US,08724,High School Not Applicable 2.0,Not Applicable,NaN,1,2.0,No,Yes,4
3,133,6,Train,Wilmington,DE,US,19802,Bachelor's Sports Management 9.0,Sports Management,2003-05-01 00:00:00,6,9.0,Yes,Yes,6
4,182,6,Train,Lenexa,KS,US,66215,High School Not Applicable 5.0,Not Applicable,2008-05-01 00:00:00,3,5.0,Yes,Yes,10


In [6]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0.0, stop_words='english')
tfidf_matrix = tf.fit_transform(users['DegreeType'])

In [35]:
# Dumping the file for the frontend access.
import joblib
joblib.dump(tf, 'tfidf_vectorizer.pkl')
joblib.dump(tfidf_matrix, 'tfidf_matrix.pkl')

['tfidf_matrix.pkl']

In [11]:
print(tfidf_matrix.shape)

(22133, 11277)


In [14]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [15]:
cosine_sim.shape

(25511, 25511)

In [24]:
cosine_sim

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 1., 1., 0.],
       ...,
       [0., 0., 1., ..., 1., 1., 0.],
       [0., 0., 1., ..., 1., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [16]:
user_based_approach = users.reset_index()
userid = user_based_approach['UserID']
indices = pd.Series(user_based_approach.index, index=user_based_approach['UserID'])

In [17]:
def get_recommendations_userwise(userid):
    idx = indices[userid]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    user_indices = [i[0] for i in sim_scores]
    user_indices_top = user_indices[1:12]
    similar_users = list(user_based_approach.loc[indices.iloc[user_indices_top]]['UserID'].values)
    return similar_users

In [25]:
print ("-----Top 10 Similar users with userId: 554------")
get_recommendations_userwise(13)

-----Top 10 Similar users with userId: 554------


[363411,
 1296070,
 465442,
 1121264,
 109317,
 1199131,
 56406,
 1209293,
 1299650,
 1306730,
 695542]

In [26]:
apps['UserID'].unique()

array([     13,      64,     101, ..., 1471948, 1472019, 1472066])

In [27]:
def get_job_id(usrid_list):
    jobs_userwise = apps['UserID'].isin(usrid_list) #
    df1 = pd.DataFrame(data = apps[jobs_userwise], columns=['JobID'])
    joblist = df1['JobID'].tolist()
    Job_list = jobs['JobID'].isin(joblist) #[1083186, 516837, 507614, 754917, 686406, 1058896, 335132])
    df_temp = pd.DataFrame(data = jobs[Job_list], columns=['JobID','Title','Description','City','State'])
    return df_temp

In [29]:
get_job_id(get_recommendations_userwise(13))

,JobID,Title,Description,City,State
1835,17497,Community Living Specialist - Direct Care Posi...,NaN,Wayne,NJ
5347,51907,Restaurant Team Member - Crew,NaN,Chevy Chase,MD
6007,58250,Physical Therapist - IU Health Morgan Hospital,NaN,Bethany,IN
6545,63095,Human Capital Principal Financial Serv Emp Ben...,NaN,Glen Allen,VA
11901,111105,Program Administrative Coordinator,NaN,Covina,CA
17311,162856,Store Manager Off-site,NaN,Rialto,CA
22560,216510,$$55K Receptionist - Flexible Hours,NaN,San Marcos,CA
23084,221335,Youth Counselor Direct Care Residential,NaN,Fitchburg,MA
26242,253478,"Specialist, Walk (Service Areas: AL & FL)",NaN,Pensacola,FL
26401,255080,State Farm Insurance and Financial Services Agent,NaN,Council Bluffs,IA
